# Preprocesamientos básicos y representación vectorial Bag-of-words aplicados al análisis de sentimientos

## 1. ¿Qué es el lenguaje humano?

- Es un proceso de __comunicación__ entre humanos basado sobre un sistema complejo de signos

- Es un proceso de __percepción__ del mundo 

<i>¿Es posible pensar el mundo fuera del lenguaje? Ver concepto de <u>neolengua</u> de George Orwell en la novela 1984, quién puede manipular el lenguaje, puede manipular los humanos.</i>

- El lenguaje humano puede ser __verbal__ o __no verbal__

- El lenguaje verbal puede ser __oral__ o __escrito__

- Los humanos utilizan el lenguaje para cumplir 6 grandes tipos de función:
    1. _expresar su subjetividad o relaciones de poder_ (opiniones, emociones, creencias, etc.). Ej: "¡Me gusta la música!", "soy mejor que tú", etc.
    1. _solicitar que el interlocutor exprese su subjetividad_. Ej: "¿Qué piensas?", "¿te "gusta el deporte?", etc.
    1. _describir el mundo_. Ej: "Hay una mesa y 4 sillas", etc.
    1. _activar o mantener la comunicación_. Ej: "Hola", "Allo", "mmm", "eeeh"
    1. _ponerse de acuerdo sobre el sentido de un signo_. Ej: "Hace frio significa que la temperatura es bajo 10°c"
    1. _comunicar por el placer de comunicar_. Ej: poesia, juegos de palabras, bromas lingúïsticas, etc.
    

- En conclusión, el lenguaje humano siempre se inscribe en una __situación de comunicación__, revela __explicatamente__ o __implicitamente__ el objetivo de comunicacion un locutor, dentro de un contexto particular. 

Ejemplo: <code>"¿Viste como llueve?"</code>

Tal vez quiere decir en realidad: "¡deberías tomar tu paragua!"

- El lenguaje humano tiene distinto nivel de interpretación:
    - nivel __lexico-semántico__ : significado las palabras utilizadas.
    - nivel __pragmático-discursivo__ : significado de estas palabras en su contexto.

<code>"Hemos realizados intervenciones quirúrgicas en afganistán"</code>

Busca significar que las intervenciones fueron las más limpias y precisas posibles, pero tambien busca ocultar que se trata de bombardeos y de violencia.
       

## 2. ¿Qué el Tratamiento Automático del Lenguaje (o _NLP_)?

- Es una sub-disciplina de la Informática y de la Inteligencia Artificial que busca dotar los computadores de capacidad para entender, traducir y generar lenguaje humano a través de algoritmos y datos.

- Es una disciplina antigua pero creciente dado el desarrollo de la comunicación en Internet y el desarrollo de las técnicas de Machine Learning/Deep Learning.

- Problemas particulares del TAL en comparación con otras areas: __datos no estructurados__, __muchas(!!!) ambiguedades en los datos__, __problemas de grandes dimensiones__ (muchas variables posibles).

- Tareas clásicas: Traducción Automática, Question-Answering, Análisis de opiniones y sentimientos, Extracción de información, Análisis del discurso, etc.

- Tareas clásicas para el lenguaje oral: reconocimiento de las palabras, reconocimiento del locutor, etc.

- Tarea para el lenguaje no verbal: reconocimiento des las emociones, reconocimiento de gestos, etc.

## 3. Preprocesamientos básicos con spaCy

### 3.1 Tokenización, Stop-Words y Lematización

Cargaremos la librería de NLP spaCy (https://spacy.io/) y los modelos para procesar textos en español.

<code>pip3 install -U spacy</code>

<code>python3 -m spacy download es_core_news_sm</code>

In [1]:
import spacy

nlp = spacy.load("es_core_news_sm")

In [2]:
doc = nlp(u"Donald Trump es el presidente de Estados Unidos.")

<img src="pipeline.png"></img>

In [3]:
for token in doc:
    print(token.text, token.is_stop, token.lemma_)
    #Stop word: "palabras gramaticales que no llevan un sentido importante"

Donald False Donald
Trump False Trump
es True ser
el True el
presidente False presidente
de True de
Estados True Estados
Unidos False Unidos
. False .


El concepto de 'stop words' no tiene una definición objetiva. Una palabra 'stop words' depende mucho del caso de uso. Habitualmente se trata de una lista de palabras gramaticales tales como "es", "el", "la", "quiere", etc.

In [ ]:
my_stop_words = [u'presidente']
for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True
    
my_non_stop_words = [u'Estados']
for nonstopword in my_non_stop_words:
    lexeme = nlp.vocab[nonstopword]
    lexeme.is_stop = False

In [ ]:
for token in doc:
    print(token.text, token.is_stop, token.lemma_)

### 3.2 Clasificación de la categoría gramatical de las palabras (Part Of Speach tagging)

In [ ]:
for token in doc:
    print(token.text, token.pos_)

### 3.3 Reconocimiento de los nombres de entidades (NER)

In [ ]:
doc = nlp(u'Amazon tiene oficinas en todos los paises de America del Sur.')

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Por defecto, spaCy utiliza define varios tipos de entidades entre los cuales:
- PERSON: personas
- ORG: organizaciones, empresas, instituciones, etc.
- GPE: paises, ciudades, regiones.
- LOC: lugares geografícos que no son paises, ciudades o regiones.
- PRODUCT: productos
- EVENT: eventos

In [ ]:
from spacy import displacy

displacy.render(doc, style="ent", jupyter=True)

Otros preprocesamientos posibles en el paquete spaCy: https://spacy.io/usage/linguistic-features
- Dependency parsing
- Sentence segmentation
- Rule-based matching

## 4. Representación vectorial del lenguaje: Bag-of-words

### 4.1 Modelo _Bag of words_

Avanzamos hacia la parte de aprendizaje automático del análisis de texto. Empezaremos a jugar un poco menos con palabras y un poco más con números. Representaremos los textos como __vectores__.

Los algoritmos de aprendizaje automático utilizan estos vectores en particular para hacer predicciones (algoritmos de aprendizaje supervisado) o agrupamiento (no supervisado).

La manera la más clásica de representar un texto como un vector es el modelo __Bag of Words__.

Empecemos con dos frases de ejemplo:

F1:<code>El gato juega con el perro</code>,
F2:<code>El perro duerme</code>
    
Si aplicamos los preprocesos que vimos antes, llegamos a:

F1:<code>gato jugar perro</code>,
F2:<code>perro dormir</code>

Si queremos representar esto como un vector, necesitaríamos primero construir nuestro vocabulario, que serían las palabras únicas que se encuentran en las oraciones. 

<code>Vocab = ['gato', 'jugar', 'dormir', 'perro']</code>

Y luego representar las frases con la frecuencia de aparición de cada palabra:

F1:<code>[1,1,0,1]</code>
F2:<code>[0,0,1,1]</code>

##### Limitaciones del modelo _Bag of words_

Como pueden darse cuenta, en el modelo Bag of words, se pierde el orden de las palabras y entonces se pierde parte del sentido del texto. Sin embargo, para muchas tareas de clasificación automática el modelo Bag of words es suficiente (ej: detección de spam).

Otra limitación de lo visto hasta ahora es que estamos suponiendo que cada palabra tiene la misma importancia para revelar el sentido del texto. Detallamos esta idea en la sección siguiente.

### 4.2 Un primer nivel de análisis: la ponderación de la importancia de las palabras (ej. TF-IDF)

TF-IDF es la abreviatura de _Term Frequency_ (frecuencia de término) - _Inverse Document Frequency_ (frecuencia de documento inversa). Ampliamente utilizado en los motores de búsqueda para encontrar documentos relevantes basados en una consulta, es un enfoque bastante intuitivo para convertir nuestras frases en vectores.

Como su nombre indica, TF-IDF trata de combinar dos tipos diferentes de información:
- la frecuencia de término (TF) es el número de veces que una palabra aparece en un documento dividido por el número de palabras en el texto. Mide la importancia local del término en el texto.

- IDF es la fracción inversa a escala logarítmica de los documentos que contienen la palabra. 
IDF(t) = log_e (total number of documents / number of documents with term t in it)


TF-IDF es simplemente el producto de estos dos factores - TF e IDF. Juntos, encapsulan más información en la representación vectorial, en lugar de limitarse a utilizar el recuento de palabras como en la representación vectorial de la bolsa de palabras. TF-IDF hace que las palabras raras sean más relevantes para representar el sentido del texto.

Si tomamos nuestras frases de ejemplo, tendriamos los vectores siguientes:
F1:<code>[0.1,0.1,0,0]</code>
F2:<code>[0,0,0.15,0]</code>

### 4.3 Un ejemplo real de representación vectorial de un dataset de documentos

#### 4.3.1 Dataset (o _Corpus_)

Utilizaremos un dataset de textos en español que corresponde a una muestra de 1.000 noticias publicadas en 2018 por el medio La Tercera (http://www.latercera.com).

In [ ]:
import pandas as pd

DATASET_CSV="sophia_latercera-1000.csv"

df = pd.read_csv(DATASET_CSV,sep='|',error_bad_lines=False,header=None)
df[0] = pd.to_datetime(df[0])

df.head(5)

In [ ]:
texts=df[[0,3]]
texts.columns = ['fecha', 'noticia']

texts.head(5)

In [ ]:
texts=texts.sort_values(by=['fecha'])
texts.head(5)

In [ ]:
for index,row in texts.iterrows():
    if index==6:
        print(row['noticia'])
        print(row['fecha'])

In [ ]:
n_texts_perMonth = texts.groupby(texts.fecha.dt.to_period("M")).count()
result=n_texts_perMonth['noticia']

result

In [ ]:
import matplotlib.pyplot as plt

ax = result.plot.bar(x='fecha', y='noticia', rot=90, color=(0.2, 0.4, 0.6, 0.6))

#### 4.3.2 Preprocesamientos básicos (spaCy)

In [ ]:
dataset=texts.head(5)
dataset

In [ ]:
import spacy
nlp = spacy.load("es_core_news_sm")

noticias = []

for index,row in dataset.iterrows():
    noticia = []
    doc=nlp(row['noticia'].lower())
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.is_digit and not token.like_num:
            noticia.append(token.lemma_)
    noticias.append(noticia)
    
print(noticias)

In [ ]:
noticias[2]

#### 4.3.3 Representación vectorial (Gensim): Bag of words (sin ponderación)

Gensim: https://radimrehurek.com/gensim/


In [ ]:
from gensim import corpora

dictionary = corpora.Dictionary(noticias)
print(dictionary.token2id)
#asigna un id por cada palabra del vocabulario

In [ ]:
#transformamos el dataset en un representacion vectorial tipo bag of word
dataset_vectorized = [dictionary.doc2bow(noticia) for noticia in noticias]

In [ ]:
print(dataset_vectorized[3])

#### 4.3.4 Representación vectorial (Gensim): Bag of words (con ponderación TF-IDF)

In [ ]:
#Convertir el BOW en una representación TF-IDF
from gensim import models
tfidf = models.TfidfModel(dataset_vectorized)

In [ ]:
tfidf[dataset_vectorized][3]

### 5. Aplicación en Análisis de sentimientos 

Empezaremos importando las librerías que necesitaremos para esta tarea. Ya hemos importado spaCy, pero también necesitaremos pandas y scikit-learn.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

#### 5.1 Dataset
Vamos a usar un conjunto de datos real: un conjunto de reseñas de productos de Amazon Alexa. Este conjunto de datos viene como un archivo separado por tabulaciones (.tsv). Tiene cinco columnas: 
- __rating__: se refiere a la calificación que cada usuario dio a Alexa (de 0 a 5). 
- __fecha__: fecha de la reseña
- __variación__: describe el modelo de producto Alexa que el usuario comentó.
- __verified_reviews__: contiene el texto del comentario.
- __feedback__: contiene un label, 0 o 1, que indica el sentimiento general negativo (0) o positivo (1).

In [ ]:
# Loading TSV file
df_amazon = pd.read_csv ("amazon_alexa.tsv", sep="\t")

In [ ]:
# Top 5 records
df_amazon

In [ ]:
# shape of dataframe
df_amazon.shape

In [ ]:
# View data information
df_amazon.info()

In [ ]:
# Feedback Value count
df_amazon.feedback.value_counts()

#### 5.2 Preprocesamientos y representación vectorial:

Crearemos una función personalizada <code>spacy_tokenizer()</code> que acepta una frase como entrada y la procesa en tokens, realizando lemmatización, minúsculas y eliminando palabras stop-words.

In [ ]:
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
stop_words=""

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

#### Vectorización de los textos en BoW o TF-IDF, con scikit-learn

Podemos generar una matriz BoW para nuestros datos de texto usando la clase <code>CountVectorizer</code> de scikit-learn. En el código de abajo, le decimos a CountVectorizer que use la función personalizada spacy_tokenizer que construimos como su tokenizer, y que defina el rango de ngramo que queremos.

Los N-gramos son combinaciones de palabras adyacentes en un texto dado, donde _n_ es el número de palabras que se incluyen en las fichas. Por ejemplo, en la frase "¿Quién ganará la Copa del Mundo de fútbol en 2022? Bigramas sería una secuencia de dos palabras contiguas como "quién ganará", "ganará la", y así sucesivamente. Así que el parámetro ngram_range que usaremos en el código de abajo establece los límites inferior y superior de nuestros ngramas (usaremos unigramas). Entonces asignaremos los ngramas a bow_vector.

In [ ]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
bow_vector

Podriamos también transformar los textos en vectores para tener los pesos TF-IDF de cada palabra en cada documento:

In [ ]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

#### Partición de los datos en conjuntos de entrenamiento y test para entrenar y evaluar un modelo predictivo

Usaremos la mitad de nuestro conjunto de datos como nuestro conjunto de entrenamiento, que incluirá las respuestas correctas. Luego probaremos nuestro modelo usando la otra mitad del conjunto de datos sin darle las respuestas, para ver con qué precisión funciona.

Convenientemente, scikit-learn nos da una función incorporada para hacer esto: train_test_split(). Sólo necesitamos decirle el conjunto de características que queremos que se divida (X), las etiquetas contra las que queremos que se realice la prueba (ylabels), y el tamaño que queremos usar para el conjunto de pruebas (representado como un porcentaje en forma decimal).

In [ ]:
from sklearn.model_selection import train_test_split

X = df_amazon['verified_reviews'] # the features we want to analyze
ylabels = df_amazon['feedback'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.5)


#### Creación de un pipeline y generación del modelo

Es el momento de construir nuestro modelo predictivo. Empezaremos importando el módulo LogisticRegression y creando un objeto clasificador LogisticRegression.

__Para revisar sobre qué aprende y cómo aprendre el algorítmo de regresión logística: [slides](https://docs.google.com/presentation/d/11O3ud6ywHuaro6OemhyeH07nuJtdc4ybMuTJicnMnm8/edit?usp=sharing)__

Luego, crearemos un pipeline de procesamiento con dos componentes: un vectorizador y algoritmo de clasificación basado en la regresión logística. El vectorizador utiliza preprocesamientos (spacy) y vectorización (scikit-learn) para crear una matriz para representar nuestros textos.

Una vez que se construya este pipeline, se aprende el modelo predictivo llamando el método <code>fit()</code>.

In [ ]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
modelLR = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([('preprocessing', bow_vector),
                 ('regression-ML', modelLR)])

# model generation
pipe.fit(X_train,y_train)

#### 5.3 Evaluación del modelo

Podemos evaluar el rendimiento de nuestro modelo usando el módulo de métricas de scikit-learn. Ahora que hemos entrenado nuestro modelo, pondremos nuestros datos de prueba a disposición para hacer predicciones. Luego usaremos varias funciones del módulo de métricas para ver la exactitud, precisión y recall de nuestro modelo.

In [ ]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)
print(predicted)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

In [ ]:
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, predicted)
print(confusion_matrix)
#Print de la matriz de confusión
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted))

In [ ]:
def printNMostInformative(vectorizer, model, N):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(model.coef_[0], feature_names))
    topClass1 = coefs_with_fns[:N]
    topClass2 = coefs_with_fns[:-(N + 1):-1]
    print("Class 1 best: ")
    for feat in topClass1:
        print(feat)
    print("Class 2 best: ")
    for feat in topClass2:
        print(feat)

In [ ]:
printNMostInformative(bow_vector, modelLR, 20)

#### 6. Ejercicio: Análisis de sentimientos en español

Análisis de sentimientos de los tweets en español sobre aerolineas: Analizar cómo los viajeros expresaron sus sentimientos.


In [ ]:
df = pd.read_csv('tweets_public.csv', encoding='utf-8', index_col='tweet_id')
df.head(5)